## Consignas del Desafío 2

### 1. Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.

Para ello se descarga el dataset `britney-spears.txt` provisto por el enlace dejado por la cátedra y se recopila un dataset propio `taylor_swift_lirics.txt` usando la API pública de Genius. A continuación se leen los archivos utilizando el caracter `/n` para separar cada oración de las canciones contenidas en el documento y conformar así un dataframe que contenga esta información. Cada oración es un *documento* en lo que respecta a la aplicación.

In [1]:
import pandas as pd
from gensim.utils import simple_preprocess

# Read datasets using newline as separator
britney_df = pd.read_csv('./songs_dataset/britney-spears.txt', sep='/n', header=None, engine='python')
britney_df.head()

taylor_df = pd.read_csv('./songs_dataset/taylor_swift_lyrics.txt', sep='/n', header=None, engine='python')
taylor_df.head()

# Count the number of documents in each dataset
print("Cantidad de documentos (Britney):", britney_df.shape[0])
print("Cantidad de documentos (Taylor):", taylor_df.shape[0])

Cantidad de documentos (Britney): 3848
Cantidad de documentos (Taylor): 3899


A continuación se arma el vocabulario para cada artista, filtrando palabras de uso común que no aportan información relevante a través del uso de la librería `nltk`. Cada palabra es llamada *token* en este contexto; y la detección de los mismos dentro de un documento es realizada por una librería propia de `tensorflow`.

In [2]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from nltk.corpus import stopwords
import nltk

# Get english stopwords from nltk
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# Define a function to tokenize text and remove stopwords
def tokenize_no_stopwords(text):
    tokens = text_to_word_sequence(text)
    return [t for t in tokens if t not in stop_words]

# Tokenize the entire datasets
britney_sentence_tokens = [tokenize_no_stopwords(row[0]) for _, row in britney_df.iterrows()]
taylor_sentence_tokens = [tokenize_no_stopwords(row[0]) for _, row in taylor_df.iterrows()]
    
# Show some example tokens
print(britney_sentence_tokens[:20])
print(taylor_sentence_tokens[:20])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JuanI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['say', 'get', 'ready', 'revolution'], ['think', 'time', 'find', 'sorta', 'solution'], ["somebody's", 'caught', 'endless', 'pollution'], ['need', 'wake', 'stop', 'living', 'illusions', 'know', 'need', 'hear'], ['somebody', 'feel'], ['wish', 'feel', 'connected'], ['wish', 'nobodies', 'neglected', 'like', 'rocket', 'baby'], ['like', 'rocket', 'take'], ['fly', 'away', 'ay', 'ay'], ['find', 'space', 'take'], ['fly', 'away', 'ay', 'ay'], ['find', 'place', 'take', 'know', 'say', 'mixing', 'races'], ['end', 'got', 'faces'], ['mama', 'told', 'got', 'love', 'first'], ['disagree', 'get', 'damn', 'earth', 'want', 'feel', 'connected'], ['want', 'neglected'], ['wish', 'find', 'places'], ['wish', 'escalate', 'yeah', 'like', 'rocket', 'baby'], ['like', 'rocket', 'take'], ['fly', 'away', 'ay', 'ay']]
[['wanted', 'wanted'], [], ['country', "lovin'", 'really', 'something'], ['three', 'months', 'half'], ['everybody', 'said', 'bad', 'news'], ['baby', 'betting'], ['guess', 'owe', 'friends', 'ten', 'dollar

Ahora para el vectorizado de los tokens de cada dataset se usa `gensim` como solicita la consigna. Esta librería provee una técnica llamada *Word2Vec* que implica utilizar un modelo de red neuronal capaz de agrupar tokens por su similitud dentro de un contexto dado. El modelo utilizado en este caso es Skip-gram debido a que permite detectar palabras poco frecuentes y darles más importancia en un contexto dado; en contraposición con un modelo CBOW que funciona más rápido pero no destaca tanto el contexto de una palabra sino las palabras con la que se suele rodear un token dado. Más información sobre el tema se puede encontrar en [el siguiente enlace](https://es.wikipedia.org/wiki/Word2vec#CBOW_y_skip-gram).

Del material de clase se recupera la sobrescritura del callback de Any2Vec para escribir el valor de la función de pérdida en cada iteración durante el entrenamiento del modelo.

In [3]:
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

# Overwrite callback to print loss after each epoch
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

# Use Word2Vec to create word embeddings. Set the model for the embedding as Skip-gram (sg=1)
britney_w2v_model = Word2Vec(min_count=4,
                     window=2,
                     vector_size=300,
                     negative=20,
                     workers=4,
                     sg=1)

taylor_w2v_model = Word2Vec(min_count=4,
                     window=2,
                     vector_size=300,
                     negative=20,
                     workers=4,
                     sg=1)

# Once the model is created, build the vocabulary with the sentence tokens
britney_w2v_model.build_vocab(britney_sentence_tokens)
taylor_w2v_model.build_vocab(taylor_sentence_tokens)

# Words in the vocabulary
print("Cantidad de words distintas en el corpus (Britney):", len(britney_w2v_model.wv.index_to_key))
print("Cantidad de words distintas en el corpus (Taylor):", len(taylor_w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus (Britney): 606
Cantidad de words distintas en el corpus (Taylor): 752


Tras crear la arquitectura de los modelos, solo resta entrenarlos para obtener una buena representación vectorizada de las palabras más similares entre sí.

In [4]:
britney_w2v_model.train(britney_sentence_tokens,
                 total_examples=britney_w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

taylor_w2v_model.train(taylor_sentence_tokens,
                 total_examples=taylor_w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 83378.09375
Loss after epoch 1: 41478.640625
Loss after epoch 2: 38622.734375
Loss after epoch 3: 38829.34375
Loss after epoch 4: 37559.796875
Loss after epoch 5: 37353.421875
Loss after epoch 6: 35987.03125
Loss after epoch 7: 35930.09375
Loss after epoch 8: 34831.78125
Loss after epoch 9: 33310.28125
Loss after epoch 10: 31710.4375
Loss after epoch 11: 30264.84375
Loss after epoch 12: 28505.03125
Loss after epoch 13: 26801.53125
Loss after epoch 14: 25542.0
Loss after epoch 15: 24037.625
Loss after epoch 16: 23207.9375
Loss after epoch 17: 22434.875
Loss after epoch 18: 20974.8125
Loss after epoch 19: 20778.75
Loss after epoch 20: 19362.75
Loss after epoch 21: 19172.125
Loss after epoch 22: 18486.9375
Loss after epoch 23: 18354.25
Loss after epoch 24: 17538.1875
Loss after epoch 25: 17394.8125
Loss after epoch 26: 16966.625
Loss after epoch 27: 16779.3125
Loss after epoch 28: 16445.875
Loss after epoch 29: 16247.5625
Loss after epoch 30: 15806.0625
Loss after epoc

(888591, 1397500)

### 2. Probar términos de interés y explicar similitudes en el espacio de embeddings

Con el modelo ya entrenado, se puede ahora buscar palabras en el vocabulario generado para cada artista y observar cuáles son las palabras más relacionadas con el término buscado (suponiendo que existan). Un ejercicio interesante es mostrar un mismo tema común a ambas artistas y observar cómo cambia el contexto que le da cada una. Se muestra por ejemplo cómo interpreta cada artista el concepto del amor.

In [5]:
# Palabras que MÁS se relacionan con...:
britney_w2v_model.wv.most_similar(positive=["love"], topn=10)

[('haha', 0.6291143894195557),
 ('rational', 0.5430219769477844),
 ('wings', 0.5192895531654358),
 ('ohhhh', 0.5174964070320129),
 ('criminal', 0.5089353919029236),
 ('physical', 0.49342045187950134),
 ('bombastic', 0.4898693561553955),
 ('sent', 0.48802709579467773),
 ('roll', 0.4836137890815735),
 ('hate', 0.4812115430831909)]

In [6]:
taylor_w2v_model.wv.most_similar(positive=["love"], topn=10)

[('mad', 0.4945010840892792),
 ('slip', 0.4479531943798065),
 ('lie', 0.4287341833114624),
 ('real', 0.4237478971481323),
 ('used', 0.41310805082321167),
 ("why'd", 0.40498632192611694),
 ('rough', 0.4026148319244385),
 ('cry', 0.3930530250072479),
 ('boarded', 0.39304324984550476),
 ('fallen', 0.39029765129089355)]

Se observa que Britney Spears utiliza el concepto del amor de una forma más pasional, destacando sensaciones y sentimientos extremos y realzados; mientras que Taylor Swift lo lleva más a contextos melancólicos, ligados a conceptos más tristes y situaciones negativas a su causa.

### 3. Graficar espacio de vectores

Finalmente, con los vectores generados puede hacerse una representación como una nube de palabras que agrupe los conceptos más usados en conjunto entre sí y separe más las palabras menos similares. Cabe destacar que, como se mencionó en clase, esta es una visualización resultado de una reducción de dimensionalidad usando PCA, por lo que la distancia entre términos tras su transformación puede no resultar tan precisa como medirla en los vectores multidimensionales que se usan originalmente; pero aún así dan una buena idea de la cercanía aparente entre términos.

Se limita el dibujo de la nube de palabras a 200 palabras y se abre el gráfico en el navegador para poder tener una interfaz más interactiva (y grande) donde evaluar los resultados.

In [7]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np
import plotly.express as px
import os

# Limiting threads to avoid MKL/OMP conflicts on Windows
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

# Function to reduce dimensions using PCA + TSNE
def reduce_dimensions(model, num_dimensions=3, pca_components=50, max_words=None):
    """
    Reduce dimensionalidad de un modelo Word2Vec a `num_dimensions` para visualización 2D o 3D.
    Primero aplica PCA a `pca_components`, luego TSNE.
    
    Args:
        model: Gensim Word2Vec entrenado
        num_dimensions: dimensiones finales (2 o 3)
        pca_components: dimensiones intermedias para PCA
        max_words: número máximo de palabras a procesar (None = todas)
        
    Returns:
        vectors_tsne: np.array de shape (n_words, num_dimensions)
        labels: lista de palabras
    """
    # Vectors and its dictionary
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    # Limit to max_words if specified
    if max_words is not None:
        vectors = vectors[:max_words]
        labels = labels[:max_words]

    # Reduce dimensionality with PCA first
    pca = IncrementalPCA(n_components=min(pca_components, vectors.shape[1]))
    vectors_pca = pca.fit_transform(vectors)

    # TSNE final
    tsne = TSNE(n_components=num_dimensions, random_state=42)
    vectors_tsne = tsne.fit_transform(vectors_pca)

    return vectors_tsne, labels

# Limit the max number of words to avoid memory issues
MAX_WORDS = 200

# Reduce dimensions for Britney model
vecs, labels = reduce_dimensions(britney_w2v_model, num_dimensions=3, max_words=MAX_WORDS)

# Use Plotly to create a 3D scatter plot
fig = px.scatter_3d(
    x=vecs[:, 0],
    y=vecs[:, 1],
    z=vecs[:, 2],
    text=labels
)
fig.update_traces(marker_size=3)

# Open on browser
fig.show(renderer="browser")

# Repeat for Taylor model
vecs, labels = reduce_dimensions(taylor_w2v_model, num_dimensions=3, max_words=MAX_WORDS)
fig = px.scatter_3d(
    x=vecs[:, 0],
    y=vecs[:, 1],
    z=vecs[:, 2],
    text=labels
)
fig.update_traces(marker_size=3)
fig.show(renderer="browser")

c:\Users\JuanI\miniconda3\envs\NLP\Lib\site-packages\threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


### 4. Conclusiones

La primera conclusión que se puede sacar es que de un vistazo a la nube de palabras generada para cada artista se puede saber mucho de los temas que tratan sus canciones e incluso cómo tratan conceptos similares. Se puede intuir también el tipo de música o los sentimientos que esperan despertar cada una.

También en función de la ubicación de las palabras se puede saber si son términos más genéricos, como temas presentes en varias de sus canciones o una temática central sobre la que hablan o se trata de términos más especializados, siendo que los primeros se encuentran en el centro de su respectiva nube y los segundos en la periferia.

Por último, se puede acompañar lo visto con técnicas complementarias de clusterización como KMeans para poder entender cómo se agrupan naturalmente las palabras dentro de ejes temáticos o incluso poder llegar a segmentar las palabras en función de álbumes, por dar un ejemplo. Como se mencionó antes, es muy importante obtener la agrupación de las palabras en el espacio multidimensional original, donde la distancia entre palabras se preserva con mayor exactitud. Se presenta un ejemplo a continuación para completar esta explicación, eligiendo al azar 6 clústers para visualizar agrupaciones de las palabras.

In [ ]:
from sklearn.cluster import KMeans

# Train a KMeans model to find clusters in the word vectors
N_CLUSTERS = 6
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init=10)
clusters = kmeans.fit_predict(vecs)

# Visualize
fig = px.scatter_3d(
    x=vecs[:, 0],
    y=vecs[:, 1],
    z=vecs[:, 2],
    text=labels,
    color=clusters.astype(str),    # color by cluster
    color_discrete_sequence=px.colors.qualitative.Set1
)
fig.update_traces(marker=dict(size=4, opacity=0.8))
fig.update_layout(title=f"KMeans clustering (k={N_CLUSTERS}) - Taylor Swift")
fig.show(renderer="browser")

c:\Users\JuanI\miniconda3\envs\NLP\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.

